<a href="https://colab.research.google.com/github/islamicity/VirtualOffice/blob/master/Snippets_Accessing_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open files from Google Drive

The example below shows how to mount your Google Drive in your virtual machine using an authorization code.

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


# Open files from GitHub

To open a file from GitHub, you can clone the repo or fetch the file directly.


If you are trying to access files from a private repo, you must use a [GitHub access token](https://help.github.com/en/articles/creating-a-personal-access-token-for-the-command-line).

In [11]:
# Clone the entire repo.
!git clone -l -s git://github.com/islamicity/PythonDataScienceHandbook.git cloned-repo

%cd cloned-repo
!ls

fatal: could not create work tree dir 'cloned-repo': Operation not supported
[Errno 2] No such file or directory: 'cloned-repo'
/gdrive
MyDrive


In [12]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file california_cities.csv: Operation not 
  0 59658    0   307    0     0   1162      0  0:00:51 --:--:--  0:00:51  1158
curl: (23) Failed writing body (0 != 307)


# Open files from your local file system

`files.upload` returns a dictionary of the files which were uploaded.
The dictionary is keyed by the file name, the value is the data which was uploaded.

In [30]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving fabfile.py to fabfile.py


OSError: ignored

# Open files from GCS with gsutil

- Use the [Cloud Resource Manager](https://cloud.google.com/resource-manager/) to create a project if you do not already have one.
- [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
- See [Google Cloud Storage (GCS) Documentation](https://cloud.google.com/storage/) for more info.


In [16]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'container-registry-16515'
!gcloud config set project {project_id}

Updated property [core/project].


In [20]:
# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://{bucket_name}/file_to_download.txt /tmp/gsutil_download.txt
  
# Print the result to make sure the transfer worked.
!cat /tmp/gsutil_download.txt

BadRequestException: 400 Invalid bucket name: '{bucket_name}'


# Open files from GCS with the Cloud Storage Python API

- Use the [Cloud Resource Manager](https://cloud.google.com/resource-manager/) to create a project if you do not already have one.
- [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
- See [Google Cloud Storage (GCS) Documentation](https://cloud.google.com/storage/) for more info.

You can also use [gsutil](https://cloud.google.com/storage/docs/gsutil) to interact with [Google Cloud Storage (GCS)](https://cloud.google.com/storage/).
This snippet is based on [a larger example](https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py).

In [23]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'container-registry-16515'

# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

from apiclient.http import MediaIoBaseDownload

with open('/tmp/gsutil_download.txt', 'wb') as f:
  # Download the file from a given Google Cloud Storage bucket.
  request = gcs_service.objects().get_media(bucket='colab-sample-bucket-7ff1ae80-584f-11ed-bd87-0242ac1c0002',
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


In [24]:
# Inspect the file we downloaded to /tmp
!cat /tmp/downloaded_from_gcs.txt

cat: /tmp/downloaded_from_gcs.txt: No such file or directory
